In [1]:
#pip install aiohttp

In [2]:
from http.client import HTTPConnection
from urllib.parse import urlparse

def site_is_online(url, timeout=2):
    """Return True if the target URL is online.

    Raise an exception otherwise.
    """
    error = Exception("unknown error")
    parser = urlparse(url)
    host = parser.netloc or parser.path.split("/")[0]
    for port in (80, 443):
        connection = HTTPConnection(host=host, port=port, timeout=timeout)
        try:
            connection.request("HEAD", "/")
            return True
        except Exception as e:
            error = e
        finally:
            connection.close()
    raise error

In [3]:
# checker.py

import asyncio
from http.client import HTTPConnection
from urllib.parse import urlparse

import aiohttp
# ...

async def site_is_online_async(url, timeout=2):
    """Return True if the target URL is online.

    Raise an exception otherwise.
    """
    error = Exception("unknown error")
    parser = urlparse(url)
    host = parser.netloc or parser.path.split("/")[0]
    for scheme in ("http", "https"):
        target_url = scheme + "://" + host
        async with aiohttp.ClientSession() as session:
            try:
                await session.head(target_url, timeout=timeout)
                return True
            except asyncio.exceptions.TimeoutError:
                error = Exception("timed out")
            except Exception as e:
                error = e
    raise error

In [4]:
import asyncio
import pathlib
import sys

async def _asynchronous_check(urls):
    async def _check(url):
        error = ""
        try:
            result = await site_is_online_async(url)
        except Exception as e:
            result = False
            error = str(e)
        display_check_result(result, url, error)

    await asyncio.gather(*(_check(url) for url in urls))

In [5]:
def display_check_result(result, url, error=""):
    """Display the result of a connectivity check."""
    print(f'The status of "{url}" is:', end=" ")
    if result:
        print('"Online!" 👍')
    else:
        print(f'"Offline?" 👎 \n  Error: "{error}"')

In [6]:
def check(urls):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # 'RuntimeError: There is no current event loop...'
        loop = None

    if loop and loop.is_running():
        print('Async event loop already running. Adding coroutine to the event loop.')
        tsk = loop.create_task(_asynchronous_check(urls))
        # ^-- https://docs.python.org/3/library/asyncio-task.html#task-object
        # Optionally, a callback function can be executed when the coroutine completes
        tsk.add_done_callback(
            lambda t: print(f'Task done with result={t.result()}  << return val of main()'))
    else:
        print('Starting new event loop')
        result = asyncio.run(_asynchronous_check(urls))

In [10]:
check(["https://gitlab.tools.russianpost.ru","https://jira.russianpost.ru","https://blablabla.russianpost.ru"])

Async event loop already running. Adding coroutine to the event loop.
The status of "https://gitlab.tools.russianpost.ru" is: "Online!" 👍
The status of "https://jira.russianpost.ru" is: "Online!" 👍
The status of "https://blablabla.russianpost.ru" is: "Offline?" 👎 
  Error: "Cannot connect to host blablabla.russianpost.ru:443 ssl:default [Name or service not known]"
Task done with result=None  << return val of main()
